# Notebook Title

## Setup Python and R environment
you can ignore this section

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse
In addition: Warning messages:
1: package ‘readr’ was built under R version 4.2.3 
2: package ‘dplyr’ was built under R version 4.2.3 
3: package ‘stringr’ was built under R version 4.2.3 


## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [4]:
%%R

# Load the mortgage_data_asian.csv

mortgage_data_asian <- read.csv('mortgage_data_asian.csv')

# Clean all the census_tract that is NA
mortgage_data_asian <- mortgage_data_asian %>% 
    filter(!is.na(census_tract))

In [5]:
%%R

# Show the first few rows of the mortgage_data_asian

head(mortgage_data_asian)

  activity_year census_tract loan_type loan_purpose loan_amount income
1          2022  36091061401         1            1       75000     NA
2          2022  36003951301         1            1      145000     NA
3          2022  36013037400         1            1       75000     NA
4          2022  36075020400         1

            1      125000     NA
5          2022  36001014803         1            1      165000     NA
6          2022  36063024001         1            1      325000     NA
  property_value applicant_race.1 co.applicant_race.1 applicant_sex
1          75000                7                   7             4
2         145000                7                   7             4
3         115000                7                   7             4
4         175000                7                   7             4
5          75000                7                   7             4
6          65000                7                   7             4
  co.applicant_sex applicant_age co.applicant_age
1                4          8888             8888
2                4          8888             8888
3                4          8888             8888
4                4          8888             8888
5                4          8888             8888
6                4          8888             8888

## 👉 Grab Census Data

1. loading the Census API key

In [6]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()


False

In [7]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("97779d587a8fdba4e4f7f7d3eec838313d5b07ab"))

Loading required package: tidycensus
To install your API key for use in future sessions, run this function with `install = TRUE`.
In addition: Warning message:
package ‘tidycensus’ was built under R version 4.2.3 


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [8]:
%%R 

# Finding the Census Varaibles for the ACS 5 year survey
# Generall you'd do this in CensusReporter, but since it's down sometimes, here it is using tidycensus's load_variables function

# get every single variable in the ACS5
all_census_vars <- load_variables(2022, "acs5", cache = TRUE) 

filtered_census_vars <- all_census_vars %>% 
    filter(grepl("median income",label, ignore.case = TRUE))   # filter to those containing "median income"
    
# write to CSV so we can view it in python
filtered_census_vars %>% 
    write_csv("filtered_census_vars.csv")

# show the first few rows
filtered_census_vars %>%
    select(-geography) %>% # remove the geography column
    print(n = 20) # print the first 20 rows

# A tibble: 46 × 3
   name         label                                                    concept
   <chr>        <chr>                                                    <chr>  
 1 B06011PR_001 Estimate!!Median income in the past 12 months --!!Total: Median…
 2 B06011PR_002 Estimate!!Median income in the past 12 months --!!Total… Median…
 3 B06011PR_003 Estimate!!Median income in the past 12 months --!!Total… Median…
 4 B06011PR_004 Estimate!!Median income in the past 12 months --!!Total… Median…
 5 B06011PR_005 Estimate!!Median income in the past 12 months --!!Total… Median…
 6 B06011_001   Estimate!!Median income in the past 12 months --!!Total: Median…
 7 B06011_002   Estimate!!Median income in the past 12 months --!!Total… Median…
 8 B06011_003   Estimate!!Median income in the past 12 months --!!Total… Median…
 9 B06011_004   Estimate!!Median income in the past 12 months --!!Total… Median…
10 B06011_005   Estimate!!Median income in the past 12 months --!!Total… Median…
11 B07011

In [17]:
%%R 
# the variable B01003_001E was selectd from the census table 
# for population, which we found in censusreporter here:
# https://censusreporter.org/tables/B01003/

# in the table below, pick the geography, the variables, and the survey you want to pull from
# see the possible values here https://walker-data.com/tidycensus/articles/basic-usage.html

# Get variable from ACS
ny_census_data <- get_acs(geography = "tract", 
                      state='NY',
                      variables = c(
                        population="B01003_001",
                        med_inc="B19013_001",
                        asian = "C02003_006", 
                        med_age = "B01002_001", 
                        med_age_asian = "B01002D_001" 
                      ), 
                      year = 2022,
                      survey="acs5",
                      geometry=F)

ny_census_data

# A tibble: 27,055 × 5
   GEOID       NAME                                     variable estimate    moe
   <chr>       <chr>                                    <chr>       <dbl>  <dbl>
 1 36001000100 Census Tract 1; Albany County; New York  med_age      30.6 9.3 e0
 2 36001000100 Census Tract 1; Albany County; New York  med_age…     35.7 3.65e1
 3 36001000100 Census Tract 1; Albany County; New York  populat…   2259   5.12e2
 4 36001000100 Census Tract 1; Albany County; New York  med_inc   44547   1.12e4
 5 36001000100 Census Tract 1; Albany County; New York  asian        30   3.4 e1
 6 36001000201 Census Tract 2.01; Albany County; New Y… med_age      39.8 8.4 e0
 7 36001000201 Census Tract 2.01; Albany County; New Y… med_age…     65.2 3.22e1
 8 36001000201 Census Tract 2.01; Albany County; New Y… populat…   2465   6.08e2
 9 36001000201 Census Tract 2.01; Albany County; New Y… med_inc   33688   1.59e4
10 36001000201 Census Tract 2.01; Albany County; New Y… asian       102   1.15e2
# ℹ 2

Getting data from the 2018-2022 5-year ACS
Using FIPS code '36' for state 'NY'


## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [18]:
%%R 

ny_census_data <- ny_census_data %>% 
  # pivot from wide to long
  pivot_wider(
    names_from=variable, 
    values_from = c(estimate, moe),
    names_glue = "{variable}_{.value}"
  )

ny_census_data

# A tibble: 5,411 × 12
   GEOID       NAME  med_age_estimate med_age_asian_estimate population_estimate
   <chr>       <chr>            <dbl>                  <dbl>               <dbl>
 1 36001000100 Cens…             30.6                   35.7                2259
 2 36001000201 Cens…             39.8                   65.2                2465
 3 36001000202 Cens…             35.3                   NA                  2374
 4 36001000301 Cens…             42.1                   28                  2837
 5 36001000302 Cens…             33.1                   31.3                3200
 6 36001000401 Cens…             62.6                   39.4                2301
 7 36001000403 Cens…             41.9                   23                  4348
 8 36001000404 Cens…             19.9                   18.9                5126
 9 36001000501 Cens…             29.6                   37.5                3471
10 36001000502 Cens…             21.8                   22.5                3788
# ℹ 5

In [19]:
%%R
ny_census_data <- ny_census_data %>%
    mutate(
        # borough = str_split(NAME, ";") %>% map_chr(3) %>% str_trim(),
        asian_pct = asian_estimate / population_estimate
    )

In [21]:
%%R -o ny_census_data

ny_census_data

# A tibble: 5,411 × 13
   GEOID       NAME  med_age_estimate med_age_asian_estimate population_estimate
   <chr>       <chr>            <dbl>                  <dbl>               <dbl>
 1 36001000100 Cens…             30.6                   35.7                2259
 2 36001000201 Cens…             39.8                   65.2                2465
 3 36001000202 Cens…             35.3                   NA                  2374
 4 36001000301 Cens…             42.1                   28                  2837
 5 36001000302 Cens…             33.1                   31.3                3200
 6 36001000401 Cens…             62.6                   39.4                2301
 7 36001000403 Cens…             41.9                   23                  4348
 8 36001000404 Cens…             19.9                   18.9                5126
 9 36001000501 Cens…             29.6                   37.5                3471
10 36001000502 Cens…             21.8                   22.5                3788
# ℹ 5

In [13]:
%%R

# Drop NAME and geometry columns
ny_census_data <- ny_census_data %>% 
    select(-c(NAME))

In [26]:
%%R

# Optionally, ensure key columns are of the same type
ny_census_data $GEOID <- as.character(ny_census_data $GEOID)
mortgage_data_asian$census_tract <- as.character(mortgage_data_asian$census_tract)

# Perform the left join
joined_data <- left_join(ny_census_data , mortgage_data_asian, by = c("GEOID" = "census_tract"))

In [27]:
%%R -o joined_data

joined_data

# A tibble: 174,066 × 25
   GEOID       NAME  med_age_estimate med_age_asian_estimate population_estimate
   <chr>       <chr>            <dbl>                  <dbl>               <dbl>
 1 36001000100 Cens…             30.6                   35.7                2259
 2 36001000100 Cens…             30.6                   35.7                2259
 3 36001000100 Cens…             30.6                   35.7                2259
 4 36001000100 Cens…             30.6                   35.7                2259
 5 36001000100 Cens…             30.6                   35.7                2259
 6 36001000100 Cens…             30.6                   35.7                2259
 7 36001000100 Cens…             30.6                   35.7                2259
 8 36001000100 Cens…             30.6                   35.7                2259
 9 36001000100 Cens…             30.6                   35.7                2259
10 36001000100 Cens…             30.6                   35.7                2259
# ℹ

In [30]:
joined_data.columns

Index(['GEOID', 'NAME', 'med_age_estimate', 'med_age_asian_estimate',
       'population_estimate', 'med_inc_estimate', 'asian_estimate',
       'med_age_moe', 'med_age_asian_moe', 'population_moe', 'med_inc_moe',
       'asian_moe', 'asian_pct', 'activity_year', 'loan_type', 'loan_purpose',
       'loan_amount', 'income', 'property_value', 'applicant_race.1',
       'co.applicant_race.1', 'applicant_sex', 'co.applicant_sex',
       'applicant_age', 'co.applicant_age'],
      dtype='object')

In [31]:
%%R

# Write the joined dataframe to a CSV file


write.csv(joined_data, "joined_data.csv", row.names = FALSE)

,GEOID,NAME,med_age_estimate,med_age_asian_estimate,population_estimate,med_inc_estimate,asian_estimate,med_age_moe,med_age_asian_moe,population_moe,...,loan_purpose,loan_amount,income,property_value,applicant_race.1,co.applicant_race.1,applicant_sex,co.applicant_sex,applicant_age,co.applicant_age
0,36001000100,Census Tract 1; Albany County; New York,30.6,35.7,2259,44547.0,30,9.3,36.5,512,...,1.0,105000.0,35.0,175000,3.0,8.0,2.0,5.0,8888,9999
1,36001000100,Census Tract 1; Albany County; New York,30.6,35.7,2259,44547.0,30,9.3,36.5,512,...,1.0,175000.0,24.0,Exempt,5.0,8.0,1.0,5.0,45-54,9999
2,36001000100,Census Tract 1; Albany County; New York,30.6,35.7,2259,44547.0,30,9.3,36.5,512,...,1.0,195000.0,118.0,Exempt,5.0,8.0,1.0,5.0,<25,9999
3,36001000100,Census Tract 1; Albany County; New York,30.6,35.7,2259,44547.0,30,9.3,36.5,512,...,1.0,175000.0,37.0,175000,3.0,8.0,2.0,5.0,25-34,9999
4,36001000100,Census Tract 1; Albany County; New York,30.6,35.7,2259,44547.0,30,9.3,36.5,512,...,1.0,155000.0,115.0,205000,6.0,8.0,1.0,5.0,35-44,9999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174061,36123150502,Census Tract 1505.02; Yates County; New York,47.8,NaN,2587,70179.0,0,2.1,NaN,281,...,1.0,645000.0,300.0,995000,5.0,8.0,1.0,5.0,35-44,9999
174062,36123150502,Census Tract 1505.02; Yates County; New York,47.8,NaN,2587,70179.0,0,2.1,NaN,281,...,1.0,1005000.0,NaN,2205000,7.0,7.0,4.0,4.0,8888,8888
174063,36123150502,Census Tract 1505.02; Yates County; New York,47.8,NaN,2587,70179.0,0,2.1,NaN,281,...,1.0,795000.0,NaN,995000,7.0,7.0,4.0,4.0,8888,8888
174064,36123150502,Census Tract 1505.02; Yates County; New York,47.8,NaN,2587,70179.0,0,2.1,NaN,281,...,1.0,405000.0,NaN,485000,7.0,7.0,4.0,4.0,8888,8888
